<a href="https://colab.research.google.com/github/K02D/HLTV-Rating-Breakdown/blob/main/HLTV_Rating_Breakdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle
!pip install eli5

     |████████████████████████████████| 112kB 7.5MB/s 


In [2]:
from google.colab import files
files.upload()
# upload kaggle.json file obtained from Kaggle by going to Account -> Create New API Token

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kirondeb","key":"c14d3d20be489c2430ecd6175fc445bb"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Get dataset from kaggle
!kaggle datasets download "mateusdmachado/csgo-professional-matches"

 91% 33.0M/36.2M [00:01<00:00, 37.3MB/s]
100% 36.2M/36.2M [00:01<00:00, 32.5MB/s]


In [5]:
!unzip csgo-professional-matches.zip players.csv

Archive:  csgo-professional-matches.zip
  inflating: players.csv             


In [6]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import eli5
from eli5.sklearn import PermutationImportance

data = pd.read_csv("players.csv")

# Remove records where the old rating was used
data_updated = data.dropna(axis=0)

# Set prediction target
y = data_updated.rating

# Set input features
features = ['kills', 'assists', 'deaths', 'flash_assists', 'kast', 'adr', 'kddiff']
X = data_updated[features]

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

# Use gradient boosting to fit a model
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
model.fit(X_train, y_train, early_stopping_rounds=5,
          eval_set=[(X_val, y_val)],
          verbose=False)

# Show relative importance of each feature in the model
perm = PermutationImportance(model, random_state=1).fit(X_val, y_val)
eli5.show_weights(perm, feature_names=X_val.columns.tolist())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


[15:37:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.6732 ± 0.0197,kddiff
0.2120 ± 0.0070,adr
0.0455 ± 0.0019,kast
0.0046 ± 0.0004,kills
0.0037 ± 0.0004,deaths
0.0017 ± 0.0002,assists
0.0003 ± 0.0001,flash_assists
